In [3]:
import requests
from datetime import datetime
import psycopg2

In [4]:
API_KEY = "7215bf43b5de4fd2a9161700c44d5ee9"
BASE_URL = "https://api.football-data.org/v4/competitions/PL/scorers"
HEADERS = {"X-Auth-Token": API_KEY}

In [6]:
# Get Recent Fixtures
url_recent = "https://api.football-data.org/v4/competitions/PL/matches?status=FINISHED"
response = requests.get(url_recent, headers=HEADERS)
data = response.json()
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="bolanlelove",
    dbname="epl"
)
cur = conn.cursor()

for match in data['matches']:
    cur.execute("""
        INSERT INTO fixtures (match_id, utc_date, home_team, away_team, status, home_score, away_score, last_update)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (match_id) DO UPDATE SET
            home_score = EXCLUDED.home_score,
            away_score = EXCLUDED.away_score,
            status = EXCLUDED.status,
            last_update = EXCLUDED.last_update;
    """, (
        match['id'],
        match['utcDate'],
        match['homeTeam']['name'],
        match['awayTeam']['name'],
        match['status'],
        match['score']['fullTime']['home'] if match['score']['fullTime']['home'] is not None else 0,
        match['score']['fullTime']['away'] if match['score']['fullTime']['away'] is not None else 0,
        datetime.now()
    ))

conn.commit()


In [7]:
# Get next fixtures
url_next = "https://api.football-data.org/v4/competitions/PL/matches?status=SCHEDULED"
response1 = requests.get(url_next, headers=HEADERS)
data1 = response.json()

for match in data1['matches']:
    cur.execute("""
        INSERT INTO fixtures (match_id, utc_date, home_team, away_team, status, home_score, away_score, last_update)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (match_id) DO UPDATE SET
            status = EXCLUDED.status,
            last_update = EXCLUDED.last_update;
    """, (
        match['id'],
        match['utcDate'],
        match['homeTeam']['name'],
        match['awayTeam']['name'],
        match['status'],
        None,  # not yet played
        None,  # not yet played
        datetime.now()
    ))

conn.commit()
cur.close()
conn.close()
